In [1]:
root = '/FileStore/tables/bernhst2'
display(dbutils.fs.ls(root))

path,name,size
dbfs:/FileStore/tables/bernhst2/customer.csv/,customer.csv/,0
dbfs:/FileStore/tables/bernhst2/customer.pq/,customer.pq/,0
dbfs:/FileStore/tables/bernhst2/customer.tbl,customer.tbl,24346208
dbfs:/FileStore/tables/bernhst2/lineitem.csv/,lineitem.csv/,0
dbfs:/FileStore/tables/bernhst2/lineitem.pq/,lineitem.pq/,0
dbfs:/FileStore/tables/bernhst2/lineitem.tbl,lineitem.tbl,759863443
dbfs:/FileStore/tables/bernhst2/order.csv/,order.csv/,0
dbfs:/FileStore/tables/bernhst2/order.pq/,order.pq/,0
dbfs:/FileStore/tables/bernhst2/orders.tbl,orders.tbl,171952252


In [2]:
# get filesize in MB
def get_file_size(path):
  file = dbutils.fs.ls(path)
  size = 0
  for subfile in file:
    size += subfile.size
  return size/1024/1024

# measure querytime
def my_query(df):
  return df.count()

def measure_performance(df):
  import time
  start = time.perf_counter()
  my_query(df)
  end = time.perf_counter()
  return (end-start)


In [3]:
df_customer = spark.read.csv(root + "/customer.tbl", header=True, sep="|", inferSchema = True)
print("Customer Schema")
df_customer.printSchema()

df_customer.agg({"custkey":"count"}).show()

Customer Schema
root
-- custkey: integer (nullable = true)
-- name: string (nullable = true)
-- address: string (nullable = true)
-- nationkey: integer (nullable = true)
-- phone: string (nullable = true)
-- acctbal: double (nullable = true)
-- mktsegment: string (nullable = true)
-- comment: string (nullable = true)

+--------------+
count(custkey)|
+--------------+
 150000|
+--------------+

In [4]:
df_order = spark.read.csv(root + "/orders.tbl", header=True, sep="|", inferSchema = True)
print("Order Schema")
df_order.printSchema()

df_order.agg({"orderkey":"count"}).show()

Order Schema
root
-- orderkey: integer (nullable = true)
-- custkey: integer (nullable = true)
-- orderstatus: string (nullable = true)
-- totalprice: double (nullable = true)
-- orderdate: timestamp (nullable = true)
-- orderpriority: string (nullable = true)
-- clerk: string (nullable = true)
-- shippriority: integer (nullable = true)
-- comment: string (nullable = true)

+---------------+
count(orderkey)|
+---------------+
 1500000|
+---------------+

In [5]:
df_lineitem = spark.read.csv(root + "/lineitem.tbl", header=True, sep="|", inferSchema = True)
print("Lineitem Schema")
df_lineitem.printSchema()

df_lineitem.agg({"orderkey":"count"}).show()

Lineitem Schema
root
-- orderkey: integer (nullable = true)
-- partkey: integer (nullable = true)
-- suppkey: integer (nullable = true)
-- linenumber: integer (nullable = true)
-- quantity: integer (nullable = true)
-- extendedprice: double (nullable = true)
-- discount: double (nullable = true)
-- tax: double (nullable = true)
-- returnflag: string (nullable = true)
-- linestatus: string (nullable = true)
-- shipdate: timestamp (nullable = true)
-- commitdate: timestamp (nullable = true)
-- receiptdate: timestamp (nullable = true)
-- shipinstruct: string (nullable = true)
-- shipmode: string (nullable = true)
-- comment: string (nullable = true)

+---------------+
count(orderkey)|
+---------------+
 6001215|
+---------------+

In [6]:
df_customer.write.format("csv").mode("overwrite").save(root + "/customer.csv", header=True)
df_customer.write.format("parquet").mode("overwrite").save(root + "/customer.pq")

df_order.write.format("csv").mode("overwrite").save(root + "/order.csv", header=True)
df_order.write.format("parquet").mode("overwrite").save(root + "/order.pq")

df_lineitem.write.format("csv").mode("overwrite").save(root + "/lineitem.csv", header=True)
df_lineitem.write.format("parquet").mode("overwrite").save(root + "/lineitem.pq")


In [7]:
import pandas as pd

csv_sizes = [get_file_size(root + "/customer.csv"), get_file_size(root + "/order.csv"), get_file_size(root + "/lineitem.csv")]
pq_sizes = [get_file_size(root + "/customer.pq"), get_file_size(root + "/order.pq"), get_file_size(root + "/lineitem.pq")]

sizes = {'table': ["customer", "order", "lineitem"],'csv': csv_sizes, 'pq': pq_sizes}
df_sizes = pd.DataFrame(data=sizes)
display(df_sizes)
# sizes given in MB

table,csv,pq
customer,23.184606552124023,11.940524101257324
order,182.54938793182373,53.36775588989258
lineitem,955.6707105636597,198.63834953308105


In [8]:
# csv join
csv_customer = spark.read.csv(root + "/customer.csv", header=True, sep=",", inferSchema = True) 
csv_order = spark.read.csv(root + "/order.csv", header=True, sep=",", inferSchema = True) 
csv_lineitem = spark.read.csv(root + "/lineitem.csv", header=True, sep=",", inferSchema = True)

csv_join = csv_customer.join(csv_order, "custkey").join(csv_lineitem, "orderkey")
csv_join.explain(True)


== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(orderkey))
:- Project [custkey#198, name#199, address#200, nationkey#201, phone#202, acctbal#203, mktsegment#204, comment#205, orderkey#224, orderstatus#226, totalprice#227, orderdate#228, orderpriority#229, clerk#230, shippriority#231, comment#232]
: +- Join Inner, (custkey#198 = custkey#225)
: :- Relation[custkey#198,name#199,address#200,nationkey#201,phone#202,acctbal#203,mktsegment#204,comment#205] csv
: +- Relation[orderkey#224,custkey#225,orderstatus#226,totalprice#227,orderdate#228,orderpriority#229,clerk#230,shippriority#231,comment#232] csv
+- Relation[orderkey#252,partkey#253,suppkey#254,linenumber#255,quantity#256,extendedprice#257,discount#258,tax#259,returnflag#260,linestatus#261,shipdate#262,commitdate#263,receiptdate#264,shipinstruct#265,shipmode#266,comment#267] csv

== Analyzed Logical Plan ==
orderkey: int, custkey: int, name: string, address: string, nationkey: int, phone: string, acctbal: double, mktsegment: string, comment: string, orderstatus: string, totalprice: double, orderdate: timestamp, orderpriority: string, clerk: string, shippriority: int, comment: string, partkey: int, suppkey: int, linenumber: int, quantity: int, extendedprice: double, discount: double, tax: double, returnflag: string, ... 7 more fields
Project [orderkey#224, custkey#198, name#199, address#200, nationkey#201, phone#202, acctbal#203, mktsegment#204, comment#205, orderstatus#226, totalprice#227, orderdate#228, orderpriority#229, clerk#230, shippriority#231, comment#232, partkey#253, suppkey#254, linenumber#255, quantity#256, extendedprice#257, discount#258, tax#259, returnflag#260, ... 7 more fields]
+- Join Inner, (orderkey#224 = orderkey#252)
 :- Project [custkey#198, name#199, address#200, nationkey#201, phone#202, acctbal#203, mktsegment#204, comment#205, orderkey#224, orderstatus#226, totalprice#227, orderdate#228, orderpriority#229, clerk#230, shippriority#231, comment#232]
 : +- Join Inner, (custkey#198 = custkey#225)
 : :- Relation[custkey#198,name#199,address#200,nationkey#201,phone#202,acctbal#203,mktsegment#204,comment#205] csv
 : +- Relation[orderkey#224,custkey#225,orderstatus#226,totalprice#227,orderdate#228,orderpriority#229,clerk#230,shippriority#231,comment#232] csv
 +- Relation[orderkey#252,partkey#253,suppkey#254,linenumber#255,quantity#256,extendedprice#257,discount#258,tax#259,returnflag#260,linestatus#261,shipdate#262,commitdate#263,receiptdate#264,shipinstruct#265,shipmode#266,comment#267] csv

== Optimized Logical Plan ==
Project [orderkey#224, custkey#198, name#199, address#200, nationkey#201, phone#202, acctbal#203, mktsegment#204, comment#205, orderstatus#226, totalprice#227, orderdate#228, orderpriority#229, clerk#230, shippriority#231, comment#232, partkey#253, suppkey#254, linenumber#255, quantity#256, extendedprice#257, discount#258, tax#259, returnflag#260, ... 7 more fields]
+- Join Inner, (orderkey#224 = orderkey#252)
 :- Project [custkey#198, name#199, address#200, nationkey#201, phone#202, acctbal#203, mktsegment#204, comment#205, orderkey#224, orderstatus#226, totalprice#227, orderdate#228, orderpriority#229, clerk#230, shippriority#231, comment#232]
 : +- Join Inner, (custkey#198 = custkey#225)
 : :- Filter isnotnull(custkey#198)
 : : +- Relation[custkey#198,name#199,address#200,nationkey#201,phone#202,acctbal#203,mktsegment#204,comment#205] csv
 : +- Filter (isnotnull(custkey#225) && isnotnull(orderkey#224))
 : +- Relation[orderkey#224,custkey#225,orderstatus#226,totalprice#227,orderdate#228,orderpriority#229,clerk#230,shippriority#231,comment#232] csv
 +- Filter isnotnull(orderkey#252)
 +- Relation[orderkey#252,partkey#253,suppkey#254,linenumber#255,quantity#256,extendedprice#257,discount#258,tax#259,returnflag#260,linestatus#261,shipdate#262,commitdate#263,receiptdate#264,shipinstruct#265,shipmode#266,comment#267] csv

== Physical Plan ==
*(5) Project [orderkey#224, custkey#198, name#199, address#200, nationkey#201, phone#202, acctbal#

In [9]:
# parquet join
pq_customer = spark.read.parquet(root + "/customer.pq") 
pq_order = spark.read.parquet(root + "/order.pq") 
pq_lineitem = spark.read.parquet(root + "/lineitem.pq")

pq_join = pq_customer.join(pq_order, "custkey").join(pq_lineitem, "orderkey")
pq_join.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(orderkey))
:- Project [custkey#348, name#349, address#350, nationkey#351, phone#352, acctbal#353, mktsegment#354, comment#355, orderkey#364, orderstatus#366, totalprice#367, orderdate#368, orderpriority#369, clerk#370, shippriority#371, comment#372]
: +- Join Inner, (custkey#348 = custkey#365)
: :- Relation[custkey#348,name#349,address#350,nationkey#351,phone#352,acctbal#353,mktsegment#354,comment#355] parquet
: +- Relation[orderkey#364,custkey#365,orderstatus#366,totalprice#367,orderdate#368,orderpriority#369,clerk#370,shippriority#371,comment#372] parquet
+- Relation[orderkey#382,partkey#383,suppkey#384,linenumber#385,quantity#386,extendedprice#387,discount#388,tax#389,returnflag#390,linestatus#391,shipdate#392,commitdate#393,receiptdate#394,shipinstruct#395,shipmode#396,comment#397] parquet

== Analyzed Logical Plan ==
orderkey: int, custkey: int, name: string, address: string, nationkey: int, phone: string, acctbal: double, mktsegment: string, comment: string, orderstatus: string, totalprice: double, orderdate: timestamp, orderpriority: string, clerk: string, shippriority: int, comment: string, partkey: int, suppkey: int, linenumber: int, quantity: int, extendedprice: double, discount: double, tax: double, returnflag: string, ... 7 more fields
Project [orderkey#364, custkey#348, name#349, address#350, nationkey#351, phone#352, acctbal#353, mktsegment#354, comment#355, orderstatus#366, totalprice#367, orderdate#368, orderpriority#369, clerk#370, shippriority#371, comment#372, partkey#383, suppkey#384, linenumber#385, quantity#386, extendedprice#387, discount#388, tax#389, returnflag#390, ... 7 more fields]
+- Join Inner, (orderkey#364 = orderkey#382)
 :- Project [custkey#348, name#349, address#350, nationkey#351, phone#352, acctbal#353, mktsegment#354, comment#355, orderkey#364, orderstatus#366, totalprice#367, orderdate#368, orderpriority#369, clerk#370, shippriority#371, comment#372]
 : +- Join Inner, (custkey#348 = custkey#365)
 : :- Relation[custkey#348,name#349,address#350,nationkey#351,phone#352,acctbal#353,mktsegment#354,comment#355] parquet
 : +- Relation[orderkey#364,custkey#365,orderstatus#366,totalprice#367,orderdate#368,orderpriority#369,clerk#370,shippriority#371,comment#372] parquet
 +- Relation[orderkey#382,partkey#383,suppkey#384,linenumber#385,quantity#386,extendedprice#387,discount#388,tax#389,returnflag#390,linestatus#391,shipdate#392,commitdate#393,receiptdate#394,shipinstruct#395,shipmode#396,comment#397] parquet

== Optimized Logical Plan ==
Project [orderkey#364, custkey#348, name#349, address#350, nationkey#351, phone#352, acctbal#353, mktsegment#354, comment#355, orderstatus#366, totalprice#367, orderdate#368, orderpriority#369, clerk#370, shippriority#371, comment#372, partkey#383, suppkey#384, linenumber#385, quantity#386, extendedprice#387, discount#388, tax#389, returnflag#390, ... 7 more fields]
+- Join Inner, (orderkey#364 = orderkey#382)
 :- Project [custkey#348, name#349, address#350, nationkey#351, phone#352, acctbal#353, mktsegment#354, comment#355, orderkey#364, orderstatus#366, totalprice#367, orderdate#368, orderpriority#369, clerk#370, shippriority#371, comment#372]
 : +- Join Inner, (custkey#348 = custkey#365)
 : :- Filter isnotnull(custkey#348)
 : : +- Relation[custkey#348,name#349,address#350,nationkey#351,phone#352,acctbal#353,mktsegment#354,comment#355] parquet
 : +- Filter (isnotnull(custkey#365) && isnotnull(orderkey#364))
 : +- Relation[orderkey#364,custkey#365,orderstatus#366,totalprice#367,orderdate#368,orderpriority#369,clerk#370,shippriority#371,comment#372] parquet
 +- Filter isnotnull(orderkey#382)
 +- Relation[orderkey#382,partkey#383,suppkey#384,linenumber#385,quantity#386,extendedprice#387,discount#388,tax#389,returnflag#390,linestatus#391,shipdate#392,commitdate#393,receiptdate#394,shipinstruct#395,shipmode#396,comment#397] parquet

== Physical Plan ==
*(5) Project [orderkey#364, custkey#348, name#349, address#350

In [10]:
# measure csv performance
csv_performance = measure_performance(csv_join)
print("It took " + str(csv_performance)  + " seconds for a csv query.")

# measure parquet performance
pq_performance = measure_performance(pq_join)
print("It took " + str(pq_performance) + " seconds for a parquet query.")

# join from parquet is more than twice as fast as join from csv

It took 5.957072090999645 seconds for a csv query.
It took 2.300712922999992 seconds for a parquet query.

In [11]:
csv_selfjoin = csv_lineitem.join(csv_lineitem, "orderkey")
pq_selfjoin = pq_lineitem.join(pq_lineitem, "orderkey")

In [12]:
# measure csv performance
csv_performance = measure_performance(csv_selfjoin)
print("It took " + str(csv_performance)  + " seconds for a csv query in self-join.")

# measure parquet performance
pq_performance = measure_performance(pq_selfjoin)
print("It took " + str(pq_performance) + " seconds for a parquet query in self-join.")

# self-join from parquet is about twice as fast as self-join from csv

It took 4.6923645870001565 seconds for a csv query in self-join.
It took 1.8574855569995634 seconds for a parquet query in self-join.

In [13]:
# small to large
csv_join_small_first = csv_customer.join(csv_order, "custkey").join(csv_lineitem, "orderkey")
pq_join_small_first = pq_customer.join(pq_order, "custkey").join(pq_lineitem, "orderkey")

# large to small
csv_join_large_first = csv_lineitem.join(csv_order, "orderkey").join(csv_customer, "custkey")
pq_join_large_first = pq_lineitem.join(pq_order, "orderkey").join(pq_customer, "custkey")

# measure csv performance
csv_perf_small_first = measure_performance(csv_join_small_first)
print("It took " + str(csv_perf_small_first)  + " seconds for a csv query with smallest joined first.")

csv_perf_large_first = measure_performance(csv_join_large_first)
print("It took " + str(csv_perf_large_first)  + " seconds for a csv query with largest joined first.")

# measure parquet performance
pq_perf_small_first = measure_performance(pq_join_small_first)
print("It took " + str(pq_perf_small_first) + " seconds for a parquet query with smallest joined first.")
pq_perf_large_first = measure_performance(pq_join_large_first)
print("It took " + str(pq_perf_large_first) + " seconds for a parquet query with largest joined first.")

# parquet is faster than csv, joining smallest first is slightly faster than joining largest first

It took 5.399411936999968 seconds for a csv query with smallest joined first.
It took 5.82929348900052 seconds for a csv query with largest joined first.
It took 1.945090731000164 seconds for a parquet query with smallest joined first.
It took 1.9585916459991495 seconds for a parquet query with largest joined first.